# Harrow-Hassidim-Lloyd (HHL) Algorithm Example

## Overview

This notebook demonstrates the implementation of the **Harrow-Hassidim-Lloyd (HHL) algorithm**, a quantum algorithm for solving systems of linear equations of the form $A\mathbf{x} = \mathbf{b}$. The HHL algorithm provides exponential speedup over classical methods for certain classes of sparse, well-conditioned matrices.

## What You'll Learn

- How to implement the HHL algorithm using qBraid Algorithms
- How to create custom Hamiltonians for the algorithm
- How to generate and analyze QASM circuits for HHL
- Understanding the quantum circuit structure of the HHL algorithm

## Algorithm Background

The HHL algorithm consists of three main steps:

1. **Quantum Phase Estimation (QPE)**: Estimates the eigenvalues of matrix $A$
2. **Controlled Rotation**: Applies rotations based on the estimated eigenvalues  
3. **Inverse QPE**: Uncomputes the phase estimation to extract the solution


### Mathematical Foundation

The HHL algorithm solves the linear system $A\mathbf{x} = \mathbf{b}$ where:
- $A$ is an $N \times N$ Hermitian matrix
- $\mathbf{b}$ is the input vector
- $\mathbf{x}$ is the solution vector

### Quantum State Representation

The algorithm encodes the solution in quantum amplitudes:
$$|\mathbf{x}\rangle = \frac{1}{\sqrt{\sum_{j=1}^N |\beta_j|^2/\lambda_j^2}} \sum_{j=1}^N \frac{\beta_j}{\lambda_j} |u_j\rangle$$

Where:
- $\lambda_j$ are eigenvalues of $A$
- $|u_j\rangle$ are eigenvectors of $A$  
- $\beta_j = \langle u_j|\mathbf{b}\rangle$ are expansion coefficients


The algorithm achieves $\mathcal{O}(\log(N) s^2 \kappa^2 / \epsilon)$ runtime complexity, where:
- $N$ is the matrix dimension
- $s$ is the sparsity of the matrix
- $\kappa$ is the condition number
- $\epsilon$ is the desired precision


## 1. Import Required Libraries

First, we import the necessary modules from qBraid Algorithms and other dependencies:


In [ ]:
# Core qBraid Algorithms imports
from qbraid_algorithms.hhl import HHLLibrary  # HHL algorithm implementation
from qbraid_algorithms.qtran import QasmBuilder  # Quantum circuit builder
from qbraid_algorithms.evolution import (
    RandomizedHamiltonian,
)  # Random Hamiltonian generator

# Additional utilities
import pyqasm  # QASM parsing and manipulation

## 2. Define Custom Hamiltonian

For this example, we'll create a custom Hamiltonian class that inherits from `RandomizedHamiltonian`. This Hamiltonian represents the matrix $A$ in our linear system $A\mathbf{x} = \mathbf{b}$.

The `RandomizedHamiltonian` class generates random Hamiltonians with controlled sparsity and spectral properties, making it ideal for demonstrating the HHL algorithm.


In [ ]:
class RandomHamiltonian(RandomizedHamiltonian):
    """
    Custom Hamiltonian for HHL Algorithm Demonstration
    The Hamiltonian represents matrix A in the linear system A|x⟩ = |b⟩
    """

    def __init__(self, *args, **kwargs):
        super().__init__(reg=2, seed=42, density=0.7, *args, **kwargs)

    def apply(self, qubits):
        """Apply the Hamiltonian evolution for time t=1.0"""
        super().apply(1.0, qubits)

    def controlled(self, qubits, control):
        """Apply controlled Hamiltonian evolution for time t=1.0"""
        super().controlled(1.0, qubits, control)

## 3. Implement the HHL Algorithm

Now we'll implement the HHL algorithm using the qBraid Algorithms framework. Here's what we'll do:

1. **Create a QASM Builder**: Initialize a quantum circuit builder with 4 qubits
2. **Import HHL Library**: Load the HHL algorithm implementation
3. **Apply HHL**: Run the algorithm with our custom Hamiltonian
4. **Generate QASM**: Convert the quantum circuit to OpenQASM format


In [ ]:
# Create a quantum circuit builder with 4 qubits total, ancilla qubit automatically allocated by algorithm
builder = QasmBuilder(qubits=4)

# Import the HHL algorithm library
hhl_lib = builder.import_library(HHLLibrary)

# Apply the HHL algorithm
# Parameters:
# - RandomHamiltonian: The matrix A in A|x⟩ = |b⟩
# - [0, 1]: Input state qubits (vector |b⟩)
# - [2, 3]: Clock register qubits for phase estimation
hhl_lib.HHL(RandomHamiltonian, [0, 1], [2, 3])

# Generate the OpenQASM 3.0 circuit
qasm_str = builder.build()


print(qasm_str)

OPENQASM 3;
include "stdgates.inc";
qubit[5] qb;
bit[5] cb;
gate RandomHam_2q_s42_d70(time) aa,ab{
	rx(1.5089459495436826 * time)  aa;
	rx(1.4992953069116235 * time)  ab;
}

gate QFT2S aa,ab{
	h aa;
	cp(pi/2) aa,ab;
	h ab;
	swap ab,aa;
}

gate P_EST_2_RandomHam aa,ab,ac,ad{
	ctrl(1) @ RandomHam_2q_s42_d70(1.0)  ac, aa, ab;
	ctrl(1) @ RandomHam_2q_s42_d70(1.0)  ad, aa, ab;
	ctrl(1) @ RandomHam_2q_s42_d70(1.0)  ad, aa, ab;
	QFT2S  ac, ad;
}

gate Pest_INV_2_RandomHam aa,ab,ac,ad{
	inv @ QFT2S  ac, ad;
	ctrl(1) @ RandomHam_2q_s42_d70(1.0)  ad, aa, ab;
	ctrl(1) @ RandomHam_2q_s42_d70(1.0)  ad, aa, ab;
	ctrl(1) @ RandomHam_2q_s42_d70(1.0)  ac, aa, ab;
}

P_EST_2_RandomHam qb[0],qb[1],qb[2],qb[3];
ctrl @ ry(pi/(2**1)) qb[2],qb[4];
ctrl @ ry(pi/(2**2)) qb[3],qb[4];
Pest_INV_2_RandomHam qb[0],qb[1],qb[2],qb[3];
cb[{4}] = measure qb[{4}];



### Circuit Components Analysis of above example

The generated circuit contains several key components:

#### 1. **Custom Hamiltonian Gates**
- `RandomHam_2q_s42_d70(time)`: Custom 2-qubit Hamiltonian
- `ctrl(1) @ RandomHam_2q_s42_d70(1.0)`: Controlled Hamiltonian evolution

#### 2. **Quantum Fourier Transform (QFT)**
- `QFT2S`: 2-qubit quantum Fourier transform gate
- `inv @ QFT2S`: Inverse QFT for uncomputation

#### 3. **Phase Estimation Gates**
- `P_EST_2_RandomHam`: Phase estimation subroutine
- `Pest_INV_2_RandomHam`: Inverse phase estimation

#### 4. **Controlled Rotations**
- `ctrl @ ry(pi/(2**1))` and `ctrl @ ry(pi/(2**2))`: Controlled Y-rotations for eigenvalue encoding

#### 5. **Measurement**
- `cb[{4}] = measure qb[{4}]`: Measurement of the ancilla qubit


## 4. Circuit Unrolling

Let's analyze the generated circuit by unrolling it to see all the individual gates. This helps us understand the complete quantum circuit structure of the HHL algorithm.


In [8]:
# Parse the QASM string and unroll all gate definitions
qasm_circuit = pyqasm.loads(qasm_str)
qasm_circuit.unroll()

print(qasm_circuit)

OPENQASM 3.0;
include "stdgates.inc";
qubit[5] qb;
bit[5] cb;
rz(1.5707963267948966) qb[0];
rx(1.5707963267948966) qb[0];
rz(3.141592653589793) qb[0];
rx(1.5707963267948966) qb[0];
rz(3.141592653589793) qb[0];
cx qb[2], qb[0];
rz(0) qb[0];
rx(1.5707963267948966) qb[0];
rz(2.387119678817952) qb[0];
rx(1.5707963267948966) qb[0];
rz(3.141592653589793) qb[0];
cx qb[2], qb[0];
rz(0) qb[0];
rx(1.5707963267948966) qb[0];
rz(3.8960656283616344) qb[0];
rx(1.5707963267948966) qb[0];
rz(1.5707963267948966) qb[0];
rz(1.5707963267948966) qb[1];
rx(1.5707963267948966) qb[1];
rz(3.141592653589793) qb[1];
rx(1.5707963267948966) qb[1];
rz(3.141592653589793) qb[1];
cx qb[2], qb[1];
rz(0) qb[1];
rx(1.5707963267948966) qb[1];
rz(2.3919450001339815) qb[1];
rx(1.5707963267948966) qb[1];
rz(3.141592653589793) qb[1];
cx qb[2], qb[1];
rz(0) qb[1];
rx(1.5707963267948966) qb[1];
rz(3.8912403070456048) qb[1];
rx(1.5707963267948966) qb[1];
rz(1.5707963267948966) qb[1];
rz(1.5707963267948966) qb[0];
rx(1.5707963267